<a href="https://colab.research.google.com/github/GiackAloZ/di-playstore/blob/master/play-store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Aloisi Giacomo (giacomo.aloisi@studio.unibo.it) Mat 0000832933_

# Progetto di DI - Google Play Store: Category Classification

Questo progetto prende in esame un dataset ottenuto dal Google Play Store, dove sono state collezionate le informazioni riguardanti piu' di 10.000 app.

Lo scopo del progetto e' quello di utilizzare semplici tecniche di NLP per classificare una app nella sua categoria (eg. Game, Social, Art & Design, etc...) a partire dal suo nome.

Ho trovato il dataset su Kaggle ed e' consultabile [qui](https://www.kaggle.com/lava18/google-play-store-apps).



## Descrizione del problema e analisi esplorativa

Si deve realizzare un modello che, dato il nome di una app, la classifichi in base alla sua categoria tra le varie disponibili (eg. Game, Social, Art & Design, etc...)

Per prima cosa, importiamo le librerie che ci serviranno.

In [ ]:
import os

import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow.keras as ks
import wordcloud
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from sklearn.base import BaseEstimator, ClassifierMixin

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
nltk.download("wordnet")

### Caricamento e pulizia del dataset

Carichiamo i dati dal csv `googleplaystore.csv` sulla repo GitHub come un dataframe pandas e diamo un'occhiata alla sua shape e alle prime 5 righe.

In [ ]:
apps_data = pd.read_csv("https://raw.githubusercontent.com/GiackAloZ/di-playstore/master/data/googleplaystore.csv")
print(apps_data.shape)
apps_data.head(5)

Ci sono numerose informazioni in questo dataset, ma le colonne che andremo a considerare sono due:

- `App`, ovvero i nomi delle app
- `Category`, la categoria a cui appartiene ogni app

Andiamo a osservare quali possono essere le categorie a cui un'applicazione puo' appartenere e la loro frequenza nel dataset.

In [ ]:
apps_data["Category"].value_counts()

C'e' una categoria che probabilemente e' stata inserita per errore, cioe' la categoria identificata con il nome "1.9", perche' ha una sola occorrenza. Andiamo a vedere di cosa si tratta.

In [ ]:
wrong_data = apps_data[apps_data["Category"] == "1.9"]
wrong_data

Sembrerebbe un dato errato, infatti ha tutte le colonne sbagliate o shiftate. Possiamo rimuoverla dal dataset.

In [ ]:
apps_data = apps_data[~apps_data.index.isin(wrong_data.index)].reset_index()

### Analisi esplorativa delle feature utilizzate

Andiamo ora a visualizzare le categorie in un grafico a barre e in un grafico a torta.

In [ ]:
apps_data["Category"].value_counts().plot.bar()

In [ ]:
plt.figure(figsize=(10,10))
apps_data["Category"].value_counts().plot.pie()

Notiamo che le categorie spaziano da molto popolate (eg. FAMILY o GAME) a scarsamente popolate (eg. COMICS o BEAUTY).
Questo potrebbe causare problemi in fase di addestramento, perche' le classi risultano abbastanza sbilanciate.

Vogliamo ora farci un'idea di come sono fatti i nomi delle app. Per fare cio', andiamo a concatenare tutti i nomi in una string `text` per poi creare una `cloudword` di tutti i nomi delle app in modo da osservare a colpo d'occhio quali sono le parole che compaiono piu' spesso.

In [ ]:
text = " ".join(apps_data["App"])

In [ ]:
wc = wordcloud.WordCloud(
    width=800, height=800,
    background_color="white", max_words=200
).generate(text)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc)

Notiamo subito che le parole piu' utilizzate sono quelle che ci potevamo aspettare, come "App" o "Mobile" oppure anche "Free".

### Correlazione tra categoria di appartenenza e nomi delle app

Andiamo ora a ottenere le top 5 parole piu' frequenti per ogni categoria.
Per farlo, raggruppiamo il dataset per categoria e concateniamo i nomi delle app.

In [ ]:
names_category = apps_data.groupby("Category")["App"].agg(" ".join)
names_category.head(5)

Poi andiamo a contare le frequenze delle parole di ogni categoria in una BagOfWords utilizzando `CountVectorizer`

In [ ]:
vect = CountVectorizer()
bow = vect.fit_transform(names_category)
bow.shape

Ok, il `CountVectorizer` ha rilevato 8714 diverse parole. Ora andiamo a ordinarle per frequenza definendo una funzione che ci restituisce le k parole piu' frequenti in una categoria.

In [ ]:
def most_k_freq(cat_freqs, vect, k=1):
    word_freqs = [(word, cat_freqs[0, i]) for word, i in vect.vocabulary_.items()]
    word_freqs_sorted = sorted(word_freqs, key=lambda x: x[1], reverse=True)
    return word_freqs_sorted[:k]

Quindi per ogni riga della BagOfWords andiamo ad estrarre le k parole piu' frequenti.
Controlliamo stampando la parole piu' frequenti della prima categoria.
Ogni elemento della lista `top_words` e' una tupla con (`word`, `frequency`).


In [ ]:
k = 5
top_words = [most_k_freq(row, vect, k) for row in bow]
print(top_words[0])

Mettiamo il risultato in un dataframe, usando un `MultiIndex` per suddividerlo meglio.
Per farlo, dobbiamo prima _flattare_ la lista di liste di tuple `top_words` per renderla una lista di liste.

In [ ]:
top_words_flatten = [[x for y in cat for x in y] for cat in top_words]

ranking_labels = np.arange(k) + 1
word_labels = ["word", "freq"]

top_words_df = pd.DataFrame(
    top_words_flatten, 
    index=names_category.index,
    columns=pd.MultiIndex.from_product(
        [ranking_labels, word_labels], names=["ranking", "word_freq"]
    )
)
top_words_df.head(5)

Vediamo, ad esmpio, che la parola piu' frequente della categoria _ART & DESIGN_ e' "coloring"

Andiamo a guardare le statistiche aggregate del dataframe appena creato.

In [ ]:
top_words_df.describe()

Vediamo come la media della parola piu' frequente per ogni categoria e' di circa 50, mentre gia' alla 5a la frequenza media e' di circa 20.

Questo nota il fatto che i nomi delle app della stessa categoria sono piu' o meno simili tra loro, o comunque hanno dei termini ricorrenti.
Quindi, abbiamo una certa correlazione tra categoria della app e nome di essa.

## Preprocessing, tokenizzazione ed estrazione delle features dai nomi delle app

Uno dei passi fondamentali in un problema di NLP e' l'estrazione delle features.

Ci sono vari modi per estrarre delle feature dal testo. Si potrebbe procedere con l;utilizzo di un modello booleano per la trasformazione di un testo in vettore booleano, ma si e' preferito usare tecniche **VSM** (vectro space model) per rappresentare il testo.

Per questa analisi, si e' deciso di utilizzare come informazione principale la frequenza delle diverse parole nei nomi delle app. Per fare cio', si utilizza una tecnica di conteggio delle frequenze del singolo sample in relazione con le frequenze di tutti i sample, cosi' da ottenere la cosidetta **term frequency-inverse document frequency** (TF-IDF). Usiamo un _transformer_ di _scikit-learn_ che permette di trasformare una sequenza di testi in una rappresentazione vettoriale usando questa tecninca (`TfidfVectorizer`)

In [ ]:
vect = TfidfVectorizer()
tokenized_app_names = vect.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect.get_feature_names())}")

Usando il tokenizer di default, abbiamo 8714 token differenti.

Possiamo ridurre un po' il numero di token anche senza cambiare il tokenizer, ma solo togliendo quei token che compaiono solo una volta in tutti in nomi delle app. Per farlo, impostiamo il parametro `min_df` (minimum document frequency) pari a 2.



In [ ]:
vect_min_2 = TfidfVectorizer(min_df=2)
tokenized_app_names_min_2 = vect_min_2.fit_transform(apps_data["App"])
print(f"Number of tokens : {len(vect_min_2.get_feature_names())}")

Gia' togliendo i token che compaiono solo una volta riduciamo abbondantemente il numero di token da 8714 a 3583 quindi di oltre 2 volte.

Andiamo ora a provare lo stemming e la lemmatizzazione. Nelle due funzioni filtriamo anche le stepwords considerate tali dalla libreria `nltk` e consideriamo solo i token che sono alfabetici.

**NB**: Si presuppone che la maggior parte del testo nella app sia in lingua inglese. In realta', nel dataset compaiono anche altre lingue, come vedremo piu' tardi.

In [ ]:
def get_tokens_no_stopwords_alpha(texts):
    tokens = nltk.tokenize.word_tokenize(texts)
    #token filtering (not stopword and alphabetic)
    return {token for token in tokens
                  if token not in nltk.corpus.stopwords.words("english")
                     and token.isalpha()}

In [ ]:
def tokenizer_stem(app_names):
    tokens = get_tokens_no_stopwords_alpha(app_names)
    #stemming
    stemmer = nltk.stem.PorterStemmer()
    stemmed_tokens = {stemmer.stem(token) for token in tokens}
    #one-char token removal
    return [token for token in stemmed_tokens]

In [ ]:
def tokenizer_lemm(app_names):
    tokens = get_tokens_no_stopwords_alpha(app_names)
    #lemmatizzazione
    lemmatizer = nltk.wordnet.WordNetLemmatizer()
    lemmatized_tokens = {lemmatizer.lemmatize(token) for token in tokens}
    #one-char token removal
    return [token for token in lemmatized_tokens]

In [ ]:
vect_stem = TfidfVectorizer(min_df=2, tokenizer=tokenizer_stem)
tokenized_app_names_stem = vect_stem.fit_transform(apps_data["App"])
print(f"Number of tokens (stem): {len(vect_stem.get_feature_names())}")

vect_lemm = TfidfVectorizer(min_df=2, tokenizer=tokenizer_lemm)
tokenized_app_names_lemm = vect_lemm.fit_transform(apps_data["App"])
print(f"Number of tokens (lem): {len(vect_lemm.get_feature_names())}")

Le feature diminuiscono, ma non significamente. Probabilmente per via del fatto che i nomi delle app sono per lo piu' nomi propri (es. Instagram, Skype, etc...) e contengono poco testo utilizzato normalmente in documenti testuali.

Comunque, andiamo a confrontare la lunghezza media dei token prima e dopo lo stemming.

In [ ]:
token_lengths = pd.Series(map(len, vect_min_2.get_feature_names()))
stem_token_lengths = pd.Series(map(len, vect_stem.get_feature_names()))

print(f"Mean length no lemming: {token_lengths.mean():.2f}")
print(f"Mean length with lemming: {stem_token_lengths.mean():.2f}")

Sono entrame molto simili, quindi significa che, anche dopo lo stemming, le parole non vengono ridotte troppo in lunghezza.

**NB**: Siccome sia lo stemming che la lemmatizzazione non sono riuscite a ridurre abbastanza lo spazio delle features, si e' deciso, in primo approcio, di non utilizzarle. Successivamente, quando si fara' un'ultima fase di tuning sui modelli piu' promettenti, si provera' anche il loro utilizzo, per vedere quanto incidano sull'accuratezza del modello.

## Generazione di diversi modelli di learning

Si procede con la generazione di diversi modelli di _supervised learning_ per la predizione della categoria dato il nome dell'app. Possiamo dire che il nostro e' un problema di **classificazione multipla**, per cui ho usato alcuni dei possibili modelli utili ad affrontare questo tipo di problemi.

### Divisione train e test set

Si procede andando a selezionare le features dal dataset (nel nostro caso solo una, il nome delle app) e il _target_ della classificazione (cioe' le categorie delle app).

Si divide il dataset in _train_ e _test_ set, usando il 90% del dataset come train set e il restante 10% come test set. Nel fare cio', si dividono i dati usando la stratificazione per classe, in modo che le classi siano bilanciate allo stesso modo sia nel train set che nel test set. Questo ci permette di ottenere una misura dell'accuratezza piu' precisa quando si andra' a testare il modello sul test set.

**NB**: non viene menzionato il validation set perche' tutta la parte sperimentale di scelta degli iperparametri nei vari modelli viene fatta attraverso _cross-fold validation_ , per cui il validation set viene ogni volta preso come parte del train set. Si noti anche che il test set, una volta diviso, non viene mai usato come parametro del training o della validation per trovare i migliori iperparametri.

In [ ]:
X = apps_data["App"]
y = apps_data["Category"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=1/10,
    stratify=y,
    random_state=42
)
print(X_train.shape, X_test.shape)

Abbiamo quindi 9756 samples dedicati al training, contro 1084 per testare il modello

Segue una lista di modelli, i quali vengono addestrati sul train set usando una _K-cross-fold validation_ stratificata (riduce il bias dato da un training con classi poco distribuite) insieme a una grid-search per trovare i migliori iperparametri per ogni modello.

I modelli presi in esame sono:

- Perceptron
- Regressione logistica
- Support-Vector Machine per classificazione (con vari kernel, anche lineari)
- Multi-layer Perceptron (con due layer nascosti)
- Rete neurale con Keras (con due layer nascosti)

Ogni modello viene successivamente testato sul test set, calcolandone l'accuratezza.
Vengono anche stampati gli iperparametri che hanno avuto un'accuratezza media migliore nella varie fold di validation.

**NB**: in tutti i modelli si utilizza il `TfidVectorizer` come calcolo del VMS con parametro `min_df=2`. Per ogni modello si imposta un `random_state` per rendere riproducibili i risultati.

In [ ]:
scores = {}
best_params = {}

### Perceptron

In [ ]:
param_perceptron = {
    "perc__penalty": ["l1", "l2"],
    "perc__alpha": np.logspace(-5, -2, num=4)
}

model_perceptron = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("perc", Perceptron(random_state=42))
])

search_perceptron = GridSearchCV(
    model_perceptron,
    param_grid=param_perceptron,
    cv=StratifiedKFold(n_splits=5),
    verbose=2,
    n_jobs=3
)
search_perceptron.fit(X_train, y_train)
scores["perceptron"] = search_perceptron.score(X_test, y_test)
scores["perceptron"]

In [ ]:
best_params["perceptron"] = search_perceptron.best_params_
best_params["perceptron"]

### Logistic regression

In [ ]:
param_logistic = [
    {
        "logreg__penalty": ["l1", "l2"],
        "logreg__C": np.logspace(0, 1, num=3)
    },
    {
        "logreg__penalty": ["elasticnet"],
        "logreg__C": np.logspace(0, 1, num=3),
        "logreg__l1_ratio": [0.1, 0.5]
    }
]

model_logistic = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("logreg", LogisticRegression(random_state=42, solver="saga", multi_class="multinomial"))
])

search_logistic = GridSearchCV(
    model_logistic,
    param_grid=param_logistic,
    cv=StratifiedKFold(n_splits=5),
    verbose=2,
    n_jobs=3
)
search_logistic.fit(X_train, y_train)
scores["logreg"] = search_logistic.score(X_test, y_test)
scores["logreg"]

In [ ]:
best_params["logreg"] = search_logistic.best_params_
best_params["logreg"]

In [ ]:
pd.DataFrame(search_logistic.cv_results_).sort_values

### SVC

In [ ]:
param_svc = [
    {
        "svc__gamma" : [0.1, 1, 5],
        "svc__C" : [1, 10],
        "svc__kernel" : ["rbf"]
    },
    {
        "svc__gamma" : [0.1, 1, 5],
        "svc__C" : [1, 10],
        "svc__kernel" : ["poly"],
        "svc__degree": [3, 5]
    },
    {
        "svc__C" : [1, 10],
        "svc__kernel" : ["linear"]
    },
]

model_scv = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("svc", SVC(random_state=42))
])

search_svc = GridSearchCV(
    model_scv,
    param_grid=param_svc,
    cv=StratifiedKFold(n_splits=5),
    verbose=2,
    n_jobs=3
)
search_svc.fit(X_train, y_train)
scores["svc"] = search_svc.score(X_test, y_test)
scores["svc"]

In [ ]:
best_params["svc"] = search_svc.best_params_
best_params["svc"]

### Multi-layer perceptron

In [ ]:
param_mlp = {
    "mlp__hidden_layer_sizes" : [(size, size//2) for size in np.logspace(4, 6, num=3, base=2, dtype=np.int)],
    "mlp__alpha" : np.logspace(-3, -1, num=3)
}

model_mlp = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("mlp", MLPClassifier(random_state=42))
])

search_mlp = GridSearchCV(
    model_mlp,
    param_grid=param_mlp,
    cv=StratifiedKFold(n_splits=5),
    verbose=2,
    n_jobs=3
)
search_mlp.fit(X_train, y_train)
scores["mlp"] = search_mlp.score(X_test, y_test)
scores["mlp"]

In [ ]:
best_params["mlp"] = search_mlp.best_params_
best_params["mlp"]

### Keras neural network

In [ ]:
class KerasModel(BaseEstimator, ClassifierMixin):
    def __init__(self,
                 hidden_layer_sizes=(64,64),
                 reg_rate=1e-4,
                 epochs=20,
                 batch_size=64):
        # label encoder (string -> int)
        self._encoder = LabelEncoder()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.reg_rate = reg_rate
        self.epochs = epochs
        self.batch_size = batch_size
        super().__init__()
    
    def fit(self, X, y, **kargs):
        # encode string classes to integers
        y_encoded = self._encoder.fit_transform(y)

        # keras model with multiple hidden layers with "relu" activation
        # and one final layer with "softmax" activation for mutli-class classification
        self._model = ks.models.Sequential([
            ks.Input(shape=X.shape[1:2])] + [     # input layer with shape equal to number of classes 
            ks.layers.Dense(size,
                            activation="relu",    # ReLU activation and weights L2 regularization
                            kernel_regularizer=ks.regularizers.l2(self.reg_rate)) for size in self.hidden_layer_sizes
            ] + [ks.layers.Dense(self._encoder.classes_.size, activation="softmax")]   # output layer proba
        )
        
        self._model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["acc"]
        )

        # show only epoch number while fitting
        self._model.fit(X.toarray(), y_encoded, batch_size=self.batch_size, epochs=self.epochs, verbose=0, **kargs)
        return self
    
    def predict(self, X):
        y_pred = self._model.predict_classes(X.toarray())
        # inverse transform classes (int -> string)
        return self._encoder.inverse_transform(y_pred)

In [ ]:
param_keras = {
    "keras__hidden_layer_sizes": [(size, size) for size in np.logspace(4, 6, num=3, base=2, dtype=np.int)],
    "keras__reg_rate": np.logspace(-4, -2, num=3),
    "keras__batch_size": np.logspace(8, 10, num=3, base=2, dtype=np.int),
    "keras__epochs": [20, 50]
}

model_keras = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("keras", KerasModel())
])

search_keras = GridSearchCV(
    model_keras,
    param_grid=param_keras,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_keras.fit(X_train, y_train)
scores["keras"] = search_keras.score(X_test, y_test)
scores["keras"]

In [ ]:
best_params["keras"] = search_keras.best_params_
best_params["keras"]

## Scelta dei modelli migliori e fine tuning

Vediamo ora le performance di tutti i modelli che sono stati addestrati

In [ ]:
scores_df = pd.DataFrame([[score for _, score in scores.items()]], index=scores.keys(), columns=["accuracy"])
scores_df

Come possiamo vedere, la `LogisticRegression` e il modello `SVC` sono i due che hanno un'accuratezza piu' alta del resto dei modelli.

### Fine tuning dei migliori due modelli

Andiamo ora a fare un po' di _fine tuning_ dei due modelli migliori. Proviamo stemming e lemmatizzazione, inoltre introduciamo anche la possibilita' di considerare _n-grammi_ di lunghezza due per il calcolo del TF-IDF.

Vediamo i risultati della cross validation di entrambi.

In [ ]:
pd.DataFrame(search_logistic.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

In [ ]:
pd.DataFrame(search_svc.cv_results_).sort_values("mean_test_score", ascending=False).head(5)

Per la `LogisticRegression` usiamo sempre la regolarizzazione _ridge_ (l2) che, insieme a _elasticnet_ , sembra dare i migliori risultati, ma a differenza di _elasticnet_ e' piu' veloce nel training.
Per il `SVC` usiamo il kernel `rbf` per lo stesso motivo.

In [ ]:
scores_ft = {}
best_params_ft = {}

In [ ]:
param_logistic = {
    "vect__tokenizer": [None, tokenizer_lemm, tokenizer_stem],
    "vect__ngram_range": [(1,1), (1,2)],
    "logreg__C": np.logspace(1,1, num=1)
}

model_logistic = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("logreg", LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", penalty="l2"))
])

search_logistic = GridSearchCV(
    model_logistic,
    param_grid=param_logistic,
    cv=StratifiedKFold(n_splits=5),
    verbose=2
)
search_logistic.fit(X_train, y_train)
scores_ft["logreg"] = search_logistic.score(X_test, y_test)
scores_ft["logreg"]

In [ ]:
best_params_ft["logreg"] = search_logistic.best_params_
best_params_ft["logreg"]

In [ ]:
param_svc = {
    "vect__tokenizer": [None, tokenizer_lemm, tokenizer_stem],
    "vect__ngram_range": [(1,1), (1,2), (1,3)],
    "svc__gamma" : np.logspace(-2, 0, num=5),
}

model_scv = Pipeline([
    ("vect", TfidfVectorizer(min_df=2)),
    ("svc", SVC(random_state=42, C=10, kernel="rbf"))
])

search_svc = GridSearchCV(
    model_scv,
    param_grid=param_svc,
    cv=StratifiedKFold(n_splits=5),
    verbose=2,
    n_jobs=3
)
search_svc.fit(X_train, y_train)
search_svc.score(X_test, y_test)

In [ ]:
search_svc.best_params_

Ricontrolliamo le performance dei due modelli scelti.

In [ ]:
scores_ft_df = pd.DataFrame([[score for _, score in scores_ft.items()]], index=scores_ft.keys(), columns=["accuracy"])
scores_ft_df

## Model evaluation

Prendiamo il modello `LogistiRegression` per valutarllo un po' piu' nello specifico. In particolare, vorremo sapere informazioni sui coefficenti delle varie parole, ovvero i pesi del modello.

Come si relazionano i vari coefficenti delle parole con le classi da predirre?
Creiamo un dataframe che metta in relazione le classi con i coefficenti delle parole. In particolare, ogni cella del dataframe avra', come intersezione di classe e parola, il peso che quella parola ha per quella particolare classe.

Il peso e' il coefficente di correlazione tra la parola e la classe (positivo -> correlazione alta diretta, negativo -> correlazione alta inversa, zero -> poca o nessuna correlazione).

In [ ]:
model_logistic = search_logistic.best_estimator_
coeffs_df = pd.DataFrame(
    model_logistic.named_steps["logreg"].coef_,
    index=model_logistic.named_steps["logreg"].classes_,
    columns=model_logistic.named_steps["vect"].get_feature_names())
coeffs_df.head(5)

Vediamo, ad esempio, i coefficenti delle parole "calculator" e "flashlight" rispettivamente nelle classi "PRODUCTIVITY" e "GAME"

In [ ]:
coeffs_df.loc["PRODUCTIVITY", "calculator"]

In [ ]:
coeffs_df.loc["GAME", "flashlight"]

Come si poteva immaginare, "calculator" e' positivamente correlata con "PRODUCTIVITY", il che significa che si un'applicazione ha nel suo nome la parola "calculator" e' probabile che sia appartenga alla classe "PRODUCTIVITY".

Viceversa, siccome "flashlight" e' negativamente correlata con "GAME", difficilmente un'applicazione che contiene questa parola verra' etichettata come "GAME".

Andiamo ora a mostrare, per ogni classe, la parola con coefficente piu' alto.

In [ ]:
best_coeffs_df = pd.concat([coeffs_df.idxmax(axis=1), coeffs_df.max(axis=1)], axis=1)
best_coeffs_df.columns = ["word", "coeff"]
best_coeffs_df

Controlliamo ora la **matrice di confusione**, che e' stata normalizzata e plottata come _heatmap_

In [ ]:
y_pred = model_logistic.predict(X_test)
conf_logistic = confusion_matrix(y_test, y_pred)
conf_logistic = conf_logistic / conf_logistic.sum(axis=1)
sns.heatmap(conf_logistic, vmin=0, vmax=1)

Per capire meglio quanto il modello sia preciso rispettivamante ad ogni classe, andiamo a stampare il `classification report`.

In [ ]:
print(classification_report(y_test, y_pred))

Estraiamo dal report le classi con le migliori e le peggiori precisioni, recall e f1-score.

In [ ]:
logistic_report_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
logistic_report_df.idxmax()

In [ ]:
logistic_report_df.idxmin()

Calcoliamo ora tutte e quattro le metriche principali delle classificazione (accuratezza, precisione, recall e f1-score).

In [ ]:
def calc_acc_prec_rec_f1(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred, normalize=True)
    prec = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    f1 = f1_score(y_test, y_pred, average="macro")
    return {
        "accuracy": acc,
        "precision": prec,
        "recall": recall,
        "f1-score": f1
    }

In [ ]:
metrics_dict = calc_acc_prec_rec_f1(y_test, y_pred)
metrics_dict

Facciamolo per ogni modello e costruiamo un dataframe con i risultati.

In [ ]:
searchs = [
    ("perceptron", search_perceptron),
    ("logreg", search_logistic),
    ("svc", search_svc),
    ("mlp", search_mlp),
    ("keras", search_keras)
]

models = [(name, gs.best_estimator) for name, gs in searchs]

metrics = [
    calc_acc_prec_rec_f1(model.predict(X_test), y_test) for _, model in models
]

names = [
    name for name, _ in models
]

model_metrics_df = pd.DataFrame(metrics, index=names)

Calcoliamo i range delle accuratezze per ogni modello con una confidenza del 95%.

In [ ]:
def confidence(acc, N, Z):
    den = (2*(N+Z**2))
    var = (Z*np.sqrt(Z**2+4*N*acc-4*N*acc**2)) / den
    a = (2*N*acc+Z**2) / den
    inf = a - var
    sup = a + var
    return (inf, sup)

In [ ]:
model_metrics_df["accuracy"].map(lambda acc: confidence(acc, len(y_test), 1.96))

In [ ]:
confidence(metrics_dict["accuracy"], len(X_test), 1.96)

In [ ]:
import random
def random_prediction(X):
    return random.choices(model_logistic.classes_, k=len(X))

In [ ]:
y_random_pred = random_prediction(X_test)
random_acc = accuracy_score(y_test, y_random_pred)
random_acc

In [ ]:
def compare_confidence(acc1, acc2, N, Z):
    var_sq = acc1 * (1 - acc1) / N + acc2 * (1 - acc2) / N
    a = abs(acc1 - acc2)
    inf = a - Z * np.sqrt(var_sq)
    sup = a + Z * np.sqrt(var_sq)
    return (inf, sup)

In [ ]:
compare_confidence(random_acc, metrics_dict["accuracy"], len(X_test), 2.56)